In [71]:
imagesToGroup = "../../../Datasets/human-art/images/real_human/drama"
folderToGroupIn = "../../../Datasets/custom/Drama/"

In [73]:
import cv2
import os
import numpy as np
import torch
import shutil
from tqdm import tqdm
from ultralytics import YOLO
from torchvision.models import vgg19, VGG19_Weights
from torchvision.transforms import transforms
from sklearn.cluster import DBSCAN


featureExtractor = vgg19(weights=VGG19_Weights.DEFAULT) # VGG
featureExtractor.eval() # VGG
'''
featureExtractor = YOLO('yolov8n.pt') # YOLOv8
'''
featureExtractor.to("cuda")

features = np.empty((0, 1000))
dir = os.listdir(imagesToGroup)
processedFiles = []
pbar = tqdm(total=len(dir))
for file in dir:
    path = os.path.join(imagesToGroup, file)
    if os.path.isfile(path):
        image = cv2.imread(path, cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if image is None:
            pbar.update()
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(224), 
            transforms.Normalize(
                [0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225]
            )
        ]) # VGG
        image = transform(image) # VGG
        '''
        image = cv2.resize(image, (640, 640)) # YOLOv8
        image = image / 255 # YOLOv8
        image = transforms.ToTensor()(image) # YOLOv8
        '''
        image = image.to("cuda")
        image = torch.stack((image,))
        imageFeatures = featureExtractor(image) # VGG
        # imageFeatures = featureExtractor.predict(source=image) # YOLOv8
        features = np.append(features, imageFeatures.detach().cpu().numpy(), axis=0)
        processedFiles.append(file)
    
    pbar.update()
pbar.close()

100%|██████████| 2000/2000 [02:19<00:00, 14.34it/s]


In [74]:
i = 0.5
step = 0.01
featuresToGroup = features
filesToGroup = processedFiles
pbar = tqdm(total=int(i/step)-1)
while i >= step and len(filesToGroup) > 0:
    group0Files = []
    group0Features = np.empty((0, 1000))
    groups = DBSCAN(eps=i, metric="cosine").fit(featuresToGroup)
    countOrphans = np.count_nonzero(groups.labels_ == -1)
    if countOrphans/len(filesToGroup) < 0.05:
        i -= step
        pbar.update()
        continue
    for index, file in enumerate(filesToGroup):
        group = int(groups.labels_[index])
        groupLabel = f"{int(i*100)}_{group}" if group != -1 else "orphans"
        if group != 0 or step >= i-step:
            sourcePath = os.path.join(imagesToGroup, file)
            targetFolder = os.path.join(folderToGroupIn, groupLabel)
            targetPath = os.path.join(folderToGroupIn, groupLabel, file)
            if not os.path.exists(targetFolder):
                os.makedirs(targetFolder)
            shutil.copy(sourcePath, targetPath)
        elif group == 0:
            group0Files.append(file)
            group0Features = np.append(group0Features, [featuresToGroup[index]], axis=0)
    filesToGroup = group0Files
    featuresToGroup = group0Features
    i -= step
    pbar.update()
pbar.close()

 88%|████████▊ | 43/49 [00:21<00:03,  1.97it/s]
